# Getting Started
## Importing Scipp

The canonical way to import scipp is as `sc`:

In [ ]:
import scipp as sc

## Physical Units

Scipp supports physical units, which can be used to define physical quantities.
We can do so, e.g., by multiplying a number with a unit:

In [ ]:
length = 1.2 * sc.Unit('m')
length

In the above HTML representation the empty tuple `()` indicates the dimensions of the variable we created.
Here it is 0-D, i.e., a single value.

The unit is not simply a string.
We can convert to a different unit and let scipp handle the conversion of the qunatity's value:

In [ ]:
length.to(unit='mm')

Similarly, operations between two quantities yield the expected unit:

In [ ]:
speed = 10. * sc.Unit('m/s')
length / speed

The unit is an invaluable safeguard against errors:

In [ ]:
length + speed  # will raise an exception

With the above we can use scipp as a safer version of a "calculator".

## Physical Constants

Scipp provides physical constants in the [scipp.constants](https://scipp.github.io/generated/modules/scipp.constants.html) module.
These are taken from SciPy but are represented as variables with a unit, for ease of use and safety.
We may, e.g., import the speed of light $c$:

In [ ]:
from scipp.constants import c

In [ ]:
c

This can be used with the quantities we defined above:

In [ ]:
length / c

## Uncertainties

Scipp operations perform linear error propagation, but **operations do not track correlations**.
Please **carefully consider** whether this is appropriate for your application! 
Otherwise the error bars that scipp computes are highly misleading as they give a false sense of security but may actually be completely wrong.
See the [uncertainties](https://pythonhosted.org/uncertainties/) Python package for a solution that *can* track correlations.

To create a quantity with variance we use the "creation function" `scalar`, which returns a variable:

In [ ]:
length = sc.scalar(50.0, variance=0.01, unit='mm')
length

Note that scipp expects and stores the *variance*, i.e., the *square* ($\sigma^2$) of the standard-deviation $\sigma$.

Computations propagate uncertainties:

In [ ]:
time = sc.scalar(5.0, variance=0.1, unit='s')
length / time

The named constants in the `constants` module do not have their variance set.
However, the `physical_constants` function can optionally return constants with their known uncertainty:

In [ ]:
from scipp.constants import physical_constants

G = physical_constants('Newtonian constant of gravitation', with_variance=True)

In [ ]:
m1 = sc.scalar(1.0, variance=1e-9, unit='kg')
m2 = sc.scalar(2.0, variance=1e-9, unit='kg')
G * m1 * m2 / (length.to(unit='m') ** 2)

## Arrays

So far we have worked with single-valued quantities (indicated by an empty shape tuple `()` above) represented by a `scipp.Variable`.
The values (and variances) of a `scipp.Variable` work much like a NumPy array.
Consider a 2-D NumPy array:

In [ ]:
import numpy as np

values = np.random.rand(3, 3)
values

We can create a `scipp.Variable` from this using `array`:

In [ ]:
var = sc.array(dims=['location', 'time'], values=values, unit='K')
var

Conceptually this is an array-valued quantity.
The `unit` property was discussed above.
The new element here is the `dims` property, which assigns labels to the axes of the NumPy array.
With a plain NumPy array it is hard to tell what the following operation does (and whether it matches the intention of the author):

In [ ]:
values[2] + values[:, 2]

Scipp's dimension labels are specified when slicing and make this clear and unambiguous:

In [ ]:
var['time', 2] + var['location', 2]

Note the automatic broadcast of the operands to a common shape.
Dimension labels are also used by other operations, clearly expressing the intent:

In [ ]:
var.mean('location')

## Arrays with coordinates

Variables can be enhanced by **coordinates**.
Each coordinate is also a variable.
A variable with associated coordinates is called **data array**:

In [ ]:
location = sc.linspace('location', 1.2, 1.3, num=4, unit='mm')
time = sc.linspace('time', 1.2, 1.3, num=3, unit='s')
data = sc.array(dims=['location', 'time'], unit='K', values=np.arange(12).reshape(4, 3))
da = sc.DataArray(data, coords={'location': location, 'time': time})
da

Data array coordinates protect against operations between incompatible data:

In [ ]:
other = da.copy()  # Copy and modify, for illustration purposes
other.coords['time'][-1] += 0.1 * sc.Unit('s')
da + other  # will raise an exception

In [ ]:
da['location', 0:2] - da.mean('location')  # ok, mean over location drops location coord

Another application of coordinates is [label-based indexing](https://scipp.github.io/user-guide/slicing.html#Label-based-indexing):

In [ ]:
start = 1.22 * sc.Unit('s')
stop = 1.4 * sc.Unit('s')
da['time', start:stop]

In addition to coordinates, data arrays also provide a dictionary of attributes and a dictionary of masks.